In [2]:
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Point,LineString
df = pd.read_csv('data/sharks_cleaned.csv')

In [3]:
import os

os.environ['PROJ_LIB'] = r"C:\Users\eleanor.sluyter-mead\AppData\Local\Continuum\anaconda3\Library\share"

In [4]:
def make_geometry(pts):
    if len(pts) == 1:
        return pts[0]
    else:
        return LineString(pts)

df['point'] = df.apply(lambda r: Point(r.longitude,r.latitude), axis=1)
sorteddf = df.sort_values(by=['name', 'datetime'])
sharkdf = sorteddf.groupby('name')['point'].apply(list).reset_index()
sharkdf['geometry'] = sharkdf.apply(lambda r: make_geometry(r.point),axis=1)

In [5]:
sharkdf.head()

,name,point,geometry
0,AB,"[POINT (-80.37539 30.49353), POINT (-80.36515 ...","LINESTRING (-80.37539 30.49353, -80.36515 30.4..."
1,ANZAC,"[POINT (113.48683 -22.84421), POINT (113.5135 ...","LINESTRING (113.48683 -22.84421, 113.5135 -22...."
2,Adelaide,"[POINT (113.71675 -23.10293), POINT (113.63252...","LINESTRING (113.71675 -23.10293, 113.632523324..."
3,Al,"[POINT (-80.23236999999999 31.92491), POINT (-...","LINESTRING (-80.23236999999999 31.92491, -80.1..."
4,Albert,"[POINT (20.53316999999998 -34.48659), POINT (2...","LINESTRING (20.53316999999998 -34.48659, 20.54..."


In [6]:
shark_gdf = gpd.GeoDataFrame(sharkdf[['name', 'geometry']])
shark_gdf.crs = {'init': 'epsg:4326'}

In [7]:
shap = folium.Map(tiles='stamenwatercolor')
folium.GeoJson(shark_gdf).add_to(shap)
shap